In [33]:
import gensim
# Check that we're using the fast C++ word2vec implementation via Cython

assert gensim.models.doc2vec.FAST_VERSION
base_data_path = "D:\Projects\Datasets"
w2vec_path = os.path.join(base_data_path, "word2vec\GoogleNews-vectors-negative300.bin.gz")
model = gensim.models.KeyedVectors.load_word2vec_format(w2vec_path, binary=True)

In [188]:
# Load the documents and build a tf-idf matrix
import pandas as pd
import os
import numpy as np

document_path = os.path.join(base_data_path, "products\strawberry.json")
all_docs = all_docs[all_docs.description.notnull()]
all_docs = all_docs[all_docs.id.notnull()]
all_docs.reset_index(inplace=True)
all_docs.describe()

In [187]:
#all_docs.iloc[999]

In [183]:
model.similar_by_word('cat', topn=5)

[('cats', 0.8099379539489746),
 ('dog', 0.7609457969665527),
 ('kitten', 0.7464984655380249),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150583267211914)]

In [177]:
def generate_averaged_sentence_vector(sentence, wv):
    v = np.zeros(300)
    for w in sentence.split():
        if w in wv:
            v += wv[w]
    return v / len(sentence)


def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

title_vectors = all_docs['description'].map(lambda title: generate_averaged_sentence_vector(title, model.wv))

In [159]:
vec_a = generate_averaged_sentence_vector("hello world", model)
vec_b = generate_averaged_sentence_vector("dr dr dr", model)
cosine_sim(vec_a, vec_b)

0.14022197143923812

In [186]:
query = "cat"


def find_top_n_docs(query: str, num_docs, vectors, w2v_model):
    query_vector = generate_averaged_sentence_vector(query, w2v_model)
    rankings = []
    i = 0
    for product_vector in vectors:
        product_similarity = cosine_sim(query_vector, product_vector)
        rankings.append([i, product_similarity])
        i += 1
        
    return_value = sorted(rankings, key=lambda ranking: -ranking[1])
    
    return return_value[:num_docs]

print(find_top_n_docs(query, 25, title_vectors, model))

result_idx = list(map(lambda item: item[0], find_top_n_docs(query, 25, title_vectors, model)))
print(result_idx)
#all_docs.iloc[result_idx]["title"]

[[5062, 0.30856972067642469], [593, 0.3080234722885381], [3810, 0.30320928367575523], [7744, 0.30280123690461319], [3811, 0.30251419308402017], [2309, 0.30121252502837897], [592, 0.30096395772790197], [4018, 0.29863675786339677], [4019, 0.29863675786339677], [4192, 0.29001321375288774], [4716, 0.28590449536992468], [5102, 0.28573854408056887], [4146, 0.28303741199266164], [4147, 0.28303741199266164], [1794, 0.2804992274904079], [755, 0.27937647322278569], [5104, 0.27798425700913082], [591, 0.27750658551620799], [2126, 0.27732629027356259], [7850, 0.27714079268746628], [1283, 0.2764406058396488], [590, 0.27603459822677301], [1140, 0.2760248634076326], [4144, 0.27587447932665959], [4732, 0.2745014601663443]]
[5062, 593, 3810, 7744, 3811, 2309, 592, 4018, 4019, 4192, 4716, 5102, 4146, 4147, 1794, 755, 5104, 591, 2126, 7850, 1283, 590, 1140, 4144, 4732]
